## Imports

In [243]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
from PIL import Image
import os
import torch.nn as nn 
import numpy as np

## Hyperparams & Data Preprocessing

In [244]:
num_epochs = 10
lr = 0.001
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_ratio, val_ratio, test_ratio = 0.7, 0.2, 0.1

In [245]:
class ImageDataset(Dataset):
    def __init__(self, folder1, folder2, transformations = None):
        self.folder1 = folder1
        self.folder2 = folder2
        self.image_files = []
        self.labels = []
        self.transformations = transformations
        
        for f in os.listdir(folder1):
            if os.path.isfile(os.path.join(folder1, f)):
                self.image_files.append(os.path.join(folder1, f))
                self.labels.append(1)
                
        for f in os.listdir(folder2):
            if os.path.isfile(os.path.join(folder2, f)):
                self.image_files.append(os.path.join(folder2, f))
                self.labels.append(0)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        image = Image.open(img_name).convert("RGB")
        label = self.labels[idx]
        
        if self.transformations:
            image = self.transformations(image)
        
        return image, label

In [246]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()           # Convert images to PyTorch tensors
])

dataset = ImageDataset(folder1='./data/Brain Tumor', folder2='./data/Healthy', transformations=transform) 

In [247]:
# Split into train, val, test (70%, 20%, 10%)

train_size = int(train_ratio * len(dataset))
val_size = int(val_ratio * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


In [248]:
# Create DataLoader objects for each subset

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [249]:
len(train_dataset), len(val_dataset), len(test_dataset) # Number of images for each split

(3220, 920, 460)

In [250]:
len(train_loader), len(val_loader), len(test_loader) # Number of batches for each loader

(202, 58, 29)

## Custom ConvNet

In [251]:
class CustomConvnet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, dilation=1)
        self.bnd1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1, dilation=1)
        self.bnd2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, dilation=1)
        self.bnd3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1, dilation=1)
        self.bnd4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1, dilation=1)
        self.bnd5 = nn.BatchNorm2d(512)
        self.relu = nn.ReLU(inplace=True)
        self.deconv1 = nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn1 = nn.BatchNorm2d(512)
        self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn2 = nn.BatchNorm2d(256)
        self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.deconv4 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.deconv5 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn5 = nn.BatchNorm2d(32)
        self.classifier = nn.Linear(524288, 2)
        
    def forward(self, x):
        x1 = self.bnd1(self.relu(self.conv1(x)))
        # Complete the forward function for the rest of the encoder
        x2 = self.bnd2(self.relu(self.conv2(x1)))
        x3 = self.bnd3(self.relu(self.conv3(x2)))
        x4 = self.bnd4(self.relu(self.conv4(x3)))
        x5 = self.bnd5(self.relu(self.conv5(x4)))

        y1 = self.bn1(self.relu(self.deconv1(x5)))
        y2 = self.bn2(self.relu(self.deconv2(y1)))
        y3 = self.bn3(self.relu(self.deconv3(y2)))
        y4 = self.bn4(self.relu(self.deconv4(y3)))
        y5 = self.bn5(self.relu(self.deconv5(y4)))
        # Complete the forward function for the rest of the decoder
        
        y6 = torch.flatten(y5, 1)
        
        # print(y6.shape)
        
        score = self.classifier(y6)

        return score 

## Training

In [252]:
model = CustomConvnet().to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [255]:
train_acc = []
train_loss = []
val_acc = []
val_loss = []


min_val_loss = np.inf

for epoch in range(num_epochs):
    num_correct = 0
    epoch_train_loss = 0
    model.train()
    num_batches = 0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)

        _, predictions = outputs.max(1)
        # print(outputs.shape)
        #print(predictions.shape)
        #print(labels.shape)
        
        # print(predictions)
        
        num_correct += (predictions == labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_train_loss += loss.item()

        num_batches = batch_idx

    train_acc.append(num_correct / num_batches)
    train_loss.append(epoch_train_loss / num_batches)


    model.eval()
    epoch_val_loss = 0
    epoch_val_correct = 0
    num_val_batches = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            
            _, predictions = outputs.max(1)
            epoch_val_correct += (predictions == labels).sum().item()
            loss = criterion(outputs, labels)
            epoch_val_loss += loss.item()

            num_val_batches += 1

    epoch_val_acc = epoch_val_correct / len(val_loader.dataset)
    epoch_val_loss /= num_val_batches

    val_loss.append(epoch_val_loss)
    val_acc.append(epoch_val_acc)

    if epoch_val_loss < min_val_loss:
      min_val_loss = epoch_val_loss
      torch.save(model.state_dict(), './models/final_model.h5')



    print('Epoch [{}/{}], Loss: {:.4f}, Val Loss: {:.4f}, Val Acc: {:.4f}'.format(epoch+1, num_epochs, train_loss[-1], val_loss[-1], val_acc[-1]))

KeyboardInterrupt: 